# solving fits in folder

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name=version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [66]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ysfitsutilpy, ysphotutilpy, ccdproc, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is being installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module photutils is installed
**** module ysfitsutilpy is installed
**** module ysphotutilpy is installed
**** module ccdproc is installed
**** module version_information is installed
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2023-11-09 21:23:28 (KST = GMT+0900) 
0 Python     3.11.5 64bit [GCC 11.2.0]
1 IPython    8.15.0
2 OS         Linux 5.15.0 88 generic x86_64 with glibc2.31
3 numpy      1.26.0
4 pandas     2.0.3
5 matplotlib 3.7.2
6 scipy      1.11.1
7 astropy    5.2.1
8 photutils  1.6.0
9 ysfitsutilpy 0.2
10 ysphotutilpy 0.1.1
11 ccdproc    2.4.0
12 version_information 1.0.4


### import modules

In [2]:
from glob import glob
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip
from ccdproc import combine, ccd_process, CCDData

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.time import Time
from astropy.visualization import ZScaleInterval, ImageNormalize
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

In [68]:
#%%
#######################################################
# read all files in base directory for processing
BASEDIR = Path("/mnt/Rdata/OBS_data") 
DOINGDIR = Path(BASEDIR / "asteroid/RiLA600_STX-16803_-_1bin")
DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))

#print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))
#######################################################

len(DOINGDIRs):  165


In [3]:
Suwon = location = EarthLocation(lon=127.005 * u.deg, lat=37.308889 * u.deg, height=101 * u.m)

In [71]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)

    DOINGDIR = DOINGDIR / _astro_utilities.reduced_dir2
    
    fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
    #print("fits_in_dir", fits_in_dir)
    print("len(fits_in_dir)", len(fits_in_dir))

    if len(fits_in_dir) == 0 :
        print(f"There is no fits fils in {DOINGDIR}")
        pass
    else : 
        print(f"Starting: {str(DOINGDIR.parts[-1])}")

        summary = yfu.make_summary(DOINGDIR/"*.fit*")
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])
        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)
        print("df_light:\n{}".format(df_light))

DOINGDIR /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin
len(fits_in_dir) 40
Starting: reduced2
All 122 keywords (guessed from /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-21-48_60sec_RiLA600_STX-16803_-20c_1bin.fits) will be loaded.


/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _ROTA1 not found for /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-41-15_120sec_RiLA600_STX-16803_-20c_1bin.fits, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _ROTA2 not found for /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-41-15_120sec_RiLA600_STX-16803_-20c_1bin.fits, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _RPIX1 not found for /mnt/Rdata/OBS_data/asteroid/RiL

len(summary): 40
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
1   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
2   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
3   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
4   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
5   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
6   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
7   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
8   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
9   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
10  /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67305600    True   
11  /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67305600    True   

In [72]:
for _, row  in df_light.iterrows():
    fpath = Path(row["file"])
    hdul = fits.open(fpath)

    SOLVE, ASTAP, LOCAL = _astro_utilities.checkPSolve(fpath)
    print(SOLVE, ASTAP, LOCAL)
    
    if SOLVE :

SyntaxError: incomplete input (2583242936.py, line 8)

In [ ]:
summary_all = pd.DataFrame()

for DOINGDIR in DOINGDIRs[:] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)

    DOINGDIR = DOINGDIR / _astro_utilities.reduced_dir2
    
    fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
    #print("fits_in_dir", fits_in_dir)
    print("len(fits_in_dir)", len(fits_in_dir))

    if len(fits_in_dir) == 0 :
        print(f"There is no fits fils in {DOINGDIR}")
        pass
    else : 
        print(f"Starting: {str(DOINGDIR.parts[-1])}")

        summary = yfu.make_summary(DOINGDIR/"*.fit*")
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])
        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)
        print("df_light:\n{}".format(df_light))

        for i, row  in df_light.iterrows():
            fpath = Path(row["file"])
            hdul = fits.open(fpath)

            SOLVE, ASTAP, LOCAL = _astro_utilities.checkPSolve(fpath)
            print(SOLVE, ASTAP, LOCAL)
            
            if SOLVE :
                w = WCS(hdul[0].header)
                t_obs = Time(hdul[0].header["DATE-OBS"]) + hdul[0].header["EXPOSURE"] * u.s / 2  # middle of observation time

                cent_coord = yfu.center_radec(ccd_or_header=hdul[0].header, center_of_image=True)
                print(f"calcualted (RA, DEC): ({cent_coord.ra}, {cent_coord.dec})")
                print(f"in header (RA, DEC): ({hdul[0].header['RA']*u.deg}, {hdul[0].header['DEC']*u.deg})")
                summary[i, "offset_RA(arcmin)"] = (cent_coord.ra - hdul[0].header['RA']*u.deg).to(u.arcmin).value
                summary[i, "offset_DEC(arcmin)"] = (cent_coord.dec - hdul[0].header['DEC']*u.deg).to(u.arcmin).value
                
                altaz = AltAz(obstime=t_obs, location=Suwon)

                cent_aa = cent_coord.transform_to(altaz)
                print(f"calculated (Az, Alt): ({cent_aa.az}, {cent_aa.alt})")
                print(f"in header (Az, Alt): ({hdul[0].header['CENTAZ ']*u.deg}, {hdul[0].header['CENTALT']*u.deg})")
                summary.loc[i, "offset_AZ(arcmin)"] = (cent_aa.az - hdul[0].header['CENTAZ']*u.deg).to(u.arcmin).value
                summary.loc[i, "offset_ALT(arcmin)"] = (cent_aa.alt - hdul[0].header['CENTALT']*u.deg).to(u.arcmin).value

        summary_all = pd.concat([summary_all, summary], axis = 0)
summary_all.to_csv(f"./{DOINGDIR.parts[-1]}_mount_offset.csv")



DOINGDIR /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin
len(fits_in_dir) 40
Starting: reduced2
All 122 keywords (guessed from /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-21-48_60sec_RiLA600_STX-16803_-20c_1bin.fits) will be loaded.


/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _ROTA1 not found for /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-41-15_120sec_RiLA600_STX-16803_-20c_1bin.fits, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _ROTA2 not found for /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin/reduced2/120LACHESIS_LIGHT_R_2023-10-10-17-41-15_120sec_RiLA600_STX-16803_-20c_1bin.fits, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/anaconda3/envs/astro_Python_env/lib/python3.11/site-packages/ysfitsutilpy/filemgmt.py:298: UserWarning: Key _RPIX1 not found for /mnt/Rdata/OBS_data/asteroid/RiL

len(summary): 40
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
1   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
2   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
3   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
4   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
5   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
6   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
7   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
8   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
9   /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67406400    True   
10  /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67305600    True   
11  /mnt/Rdata/OBS_data/asteroid/RiLA600_STX-16803...  67305600    True   

In [74]:
DOINGDIR.parts[-2]

'120LACHESIS_LIGHT_-_2023-10-10_-_RiLA600_STX-16803_-_1bin'

In [ ]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4096                                                  
NAXIS2  =                 4096                                                  
FITS-TLM= '2023-11-08T13:21:54' / UT of last modification of this FITS file     
IMAGETYP= 'LIGHT'              / Type of exposure                               
EXPOSURE=                180.0 / [s] Exposure duration                          
EXPTIME =                180.0 / [s] Exposure duration                          
DATE-LOC= '2023-10-11T03:41:24.109' / Time of observation (local)               
DATE-OBS= '2023-10-10T18:41:24.109' / ISO-8601 time of observation              
XBINNING=                    1 / X axis binning factor                          
YBINNING=                   

In [ ]:
len(hdul[0].header['COMMENT'])
for comment in hdul[0].header['COMMENT'] :
    if "Mount offset" in comment:
        print(comment)

7  Solved in 0.5 sec. Offset 0.0". Mount offset RA=-6.0', DEC=-6.4'


In [ ]:
print(hdul[0].header['RA'], hdul[0].header['DEC'])
print(hdul[0].header['CENTALT'], hdul[0].header['CENTAZ '])

29.3647159665386 15.0186729797743
50.1609574584157 246.974826672091


In [ ]:
w = WCS(hdul[0].header)
print(dir(w))

['__abstractmethods__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_all_pix2world', '_all_world2pix', '_array_converter', '_as_mpl_axes', '_denormalize_sky', '_det2im', '_fix_ctype', '_fix_scamp', '_get_components_and_classes', '_get_naxis', '_init_kwargs', '_naxis', '_normalize_sky', '_p4_pix2foc', '_pix2foc', '_pixel_bounds', '_read_d2im_old_format', '_read_det2im_kw', '_read_distortion_kw', '_read_sip_kw', '_remove_sip_kw', '_write_det2im', '_write_distortion_kw', '_write_sip_kw', 'all_pix2world', 'all_world2pix', 'array_index_to_world', 'array_index_to_world_values', 'array_shape', 'axis_correlation_mat

In [ ]:
w
#t_obs = Time(hdr["DATE-OBS"]) + hdr["EXPTIME"] * u.s / 2  # middle of observation time
t_obs = Time(hdul[0].header["DATE-OBS"]) + hdul[0].header["EXPOSURE"] * u.s / 2  # middle of observation time
t_obs

<Time object: scale='utc' format='isot' value=2023-10-10T18:42:54.109>

In [ ]:
dir(cent_coord)

['T',
 '_APPLICABLE_FUNCTIONS',
 '_METHOD_FUNCTIONS',
 '__abstractmethods__',
 '__array_function__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_apply',
 '_extra_frameattr_names',
 '_is_name',
 '_sky_coord_frame',
 'altaz',
 'apply_space_motion',
 'barycentricmeanecliptic',
 'barycentrictrueecliptic',
 'cache',
 'cartesian',
 'cirs',
 'contained_by',
 'copy',
 'custombarycentricecliptic',
 'cylindrical',
 'data',
 'dec',
 'default_differential',
 'default_representation',
 'diagonal',
 'differential_type',
 'directional_offset_by',
 'distance',
 'e

In [ ]:
cent_coord = yfu.center_radec(ccd_or_header=hdul[0].header, center_of_image=True)
print(f"calcualted (RA, DEC): ({cent_coord.ra}, {cent_coord.dec})")
print(f"in header (RA, DEC): ({hdul[0].header['RA']*u.deg}, {hdul[0].header['DEC']*u.deg})")
offset_RA = (cent_coord.ra - hdul[0].header['RA']*u.deg).to(u.arcmin)
offset_DEC = (cent_coord.dec - hdul[0].header['DEC']*u.deg).to(u.arcmin)
offset_RA, offset_DEC

calcualted (RA, DEC): (29.467512664296233 deg, 15.125798223923606 deg)
in header (RA, DEC): (29.3647159665386 deg, 15.0186729797743 deg)


(<Angle 6.16780187 arcmin>, <Angle 6.42751465 arcmin>)

In [ ]:
altaz = AltAz(obstime=t_obs, location=Suwon)

aa = cent_coord.transform_to(altaz)

print(f"calculated (Az, Alt): ({aa.az}, {aa.alt})")
print(f"in header (Az, Alt): ({hdul[0].header['CENTAZ ']*u.deg}, {hdul[0].header['CENTALT']*u.deg})")


calculated (Az, Alt): (245.88152864506057 deg, 50.44500436089755 deg)
in header (Az, Alt): (246.974826672091 deg, 50.1609574584157 deg)


In [ ]:
dir(aa)
#aa.to_string()

['T',
 '_APPLICABLE_FUNCTIONS',
 '_METHOD_FUNCTIONS',
 '__abstractmethods__',
 '__array_function__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_apply',
 '_equinox',
 '_extra_frameattr_names',
 '_is_name',
 '_sky_coord_frame',
 'alt',
 'altaz',
 'apply_space_motion',
 'az',
 'barycentricmeanecliptic',
 'barycentrictrueecliptic',
 'cache',
 'cartesian',
 'cirs',
 'contained_by',
 'copy',
 'custombarycentricecliptic',
 'cylindrical',
 'data',
 'default_differential',
 'default_representation',
 'diagonal',
 'differential_type',
 'directional_offset_b

In [ ]:
for _, row  in df_light.iterrows():

    fpath = Path(row["file"])
    hdul = fits.open(fpath)
    
    SOLVE, ASTAP, LOCAL = _astro_utilities.checkPSolve(fpath)
    print(SOLVE, ASTAP, LOCAL)
    
    if SOLVE :
        

SyntaxError: incomplete input (1589980278.py, line 10)

In [ ]:
summary.columns
summary[['CENTALT', 'CENTAZ', 'RA', 'DEC']]

,CENTALT,CENTAZ,RA,DEC
0,63.396418,219.535045,29.385819,15.025158
1,63.181489,220.297523,29.385444,15.025188
2,62.892860,221.287087,29.384814,15.025177
3,62.462977,222.694513,29.383960,15.025166
4,62.298834,223.212538,29.383690,15.025165
5,62.063033,223.939101,29.383180,15.025085
6,61.758841,224.847989,29.382730,15.025044
7,61.300410,226.161334,29.381846,15.024933
8,61.125882,226.644880,29.381531,15.024913
9,60.875888,227.322636,29.381096,15.024882


In [ ]:
summary = yfu.make_summary(DOINGDIR/"*.fit*")
print("len(summary):", len(summary))
print("summary:", summary)
#print(summary["file"][0])

All 45 keywords (guessed from /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_-_2023-08-25_-_RiLA600_STX-16803_-_1bin/M 13_LIGHT_b_2023-08-25-11-16-07_30sec_RiLA600_STX-16803_-20c_1bin.fit) will be loaded.
len(summary): 90
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
1   /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
2   /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
3   /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
4   /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
..                                                ...       ...     ...   
85  /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
86  /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
87  /mnt/Rdata/OBS_data/ccd_test_folder/M 13_LIGHT_...  33560640    True   
88  /mnt/Rd